In [ ]:
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pylab as pl
from itertools import cycle


After the fit(), the pca model exposes the singular vectors in the components_ attribute:
    

# Digits
OK - lets try something more exciting than the Iris dataset (this is taken from https://github.com/oreillymedia/t-SNE-tutorial) 


In [ ]:
from sklearn import datasets
from sklearn import manifold
import matplotlib.patheffects as PathEffects
digits = datasets.load_digits()
digits.data.shape
#print(digits['DESCR'])

from sklearn.utils.extmath import _ravel
# Random state.
RS = 20150101

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(6,3))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(digits.images[i,...])
    plt.xticks([]); plt.yticks([])
    plt.title(digits.target[i])
plt.savefig('images/digits-generated.png', dpi=150)

Now let's run the t-SNE algorithm on the dataset. It just takes one line with scikit-learn.

In [ ]:
# We first reorder the data points according to the handwritten numbers.
X = np.vstack([digits.data[digits.target==i]
               for i in range(10)])
y = np.hstack([digits.target[digits.target==i]
               for i in range(10)])
digits_proj = manifold.TSNE(random_state=RS).fit_transform(X)

Here is a utility function used to display the transformed dataset. The color of each point refers to the actual digit (of course, this information was not used by the dimensionality reduction algorithm).

In [ ]:
def scatter(x, colors):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

scatter(digits_proj, y)
plt.savefig('images/digits_tsne-generated.png', dpi=120)


Observe that the images corresponding to the different digits are clearly separated into different clusters of points.

# Topic Goal

We want to apply what you learned yesterday on **machine learning approaches to learning low-dimensional projections** to high-dimensional data from a human motion tracking example

## Human Motion capture data
Let us now look at some data from human motion capture, to see if we can usefully view this high-dimensional data in a lower dimensional space. 

Although you have a large number of measurement points on a human body, that does not mean that each one is independent of the others.


In [ ]:
# load in some motion data
from sklearn import manifold
import pandas as pd
%matplotlib inline
from sklearn.utils.extmath import _ravel
# Random state.
RS = 20150101

X = pd.read_csv('handtracker/olddata/2016-6-6-9.20/data/trackerOutput5_pincer.txt')
Xbend = pd.read_csv('handtracker/olddata/2016-6-6-9.20/data/trackerOutput12_finger_bending.txt')
X.columns = ['t']+['x','y','z']*32+['c']
Xbend.columns = ['t']+['x','y','z']*32+['c']

X['t']= pd.to_datetime(X['t'], unit='us')
Xbend['t']= pd.to_datetime(Xbend['t'], unit='us')

We can plot this in parallel coordinates, but it isn't clear what the intrinsic dimensionality of the data is. How independent are these time-series?


In [ ]:

X.plot(x='t', y=range(3,97,3),legend=False,title='z')
X.plot(x='t', y=range(2,97,3),legend=False,title='y')
X.plot(x='t', y=range(1,97,3),legend=False,title='x')

but what if we try to bring this down to two dimensions?

In [ ]:
Xbend.columns = ['t']+['x','y','z']*32+['c']
Xbend.plot(x='t', y=range(3,97,3),legend=False,title='z')
Xbend.plot(x='t', y=range(2,97,3),legend=False,title='y')
Xbend.plot(x='t', y=range(1,97,3),legend=False,title='x')
plt.show()

so far we have been looking at each hand pose as a static hand configuration, so it is not picking up change of state. What if we apply some derivative operators to the time-series?